# 1. Ответьте на вопросы

## 1.1 Что такое leave-one-out LOO(кросс-валидация по отдельным объектам)? Укажите ограничения  и сильные стороны.
LOO - выборка разделяется на `L-1` и `L` объект `l` раз.

Иными словами это частный случай k-Fold: в нём каждый фолд состоит ровно из одного семпла. Применяется когда очень мало данных.

Преимущества LOO в том, что каждый объект ровно один раз участвует в контроле, а длина обучающих подвыборок лишь на единицу меньше длины полной выборки.

Недостатком LOO является большая ресурсоёмкость, так как обучаться приходится L раз.



## 1.2. Как работает `Grid Search`, `Randomized Grid Search` и `Bayesian optimization`?

`Grid Search` - для каждого гиперпараметра фиксируется несколько значений.Затем
перебираются все комбинации значений различных гиперпараметров, на каждой из этих комбинаций модель обучается и тестируется. И
выбирается комбинация, на которой модель показывает лучшее качество.

`Randomized Grid Search` - реализует рандомизированный поиск по параметрам, где каждая настройка выбирается из распределения возможных значений параметров. Это имеет два основных преимущества перед исчерпывающим поиском:

- Количество вычислений может быть выбран независимо от количества параметров и возможных значений.

- Добавление параметров, не влияет на производительность и не снижает эффективность.

`Байесовская оптимизация` — это метод глобальной оптимизации для неизвестной функции (чёрного ящика) с шумом. Применённая к гиперпараметрической оптимизации байесовская оптимизация строит стохастическую модель функции отображения из значений гиперпараметра в целевую функцию, применённую на множестве проверки. Путём итеративного применения перспективной конфигурации гиперпараметров, основанной на текущей модели, а затем её обновления, байесовская оптимизация стремится собрать как можно больше информации об этой функции и, в частности, место оптимума. Метод пытается сбалансировать зондирование (гиперпараметры, для которых изменение наименее достоверно известно) и использование (гиперпараметры, которые, как ожидается, наиболее близки к оптимуму). На практике байесовская оптимизация показала[6][7][8][9] лучшие результаты с меньшими вычислениями по сравнению с поиском по решётке и случайным поиском ввиду возможности суждения о качестве экспериментов ещё до их выполнения.

## 1.3. Объясните классификацию методов отбора признаков. Объясните, как работает корреляция пирсона и Chi2. Объясните, как работает Lasso. Объясните, что такое важность перестановки. Познакомьтесь с SHAP

Методы выделения признаков можно разделить на несколько категорий в зависимости от их подхода и цели. Вот основные категории:

- Методы фильтрации: Эти методы отбирают признаки на основе статистических показателей или оценок, рассчитанных на основе данных, без использования конкретного алгоритма машинного обучения. В качестве примеров можно привести отбор признаков на основе корреляции, тест хи-квадрат, SelectKBest и F-значение ANOVA.

- Обертывающие методы: Обертывающие методы оценивают подмножества признаков путем обучения и тестирования определенного алгоритма машинного обучения. Они выбирают признаки, основываясь на производительности модели. Примеры: рекурсивное исключение признаков, обратное исключение признаков и прямой отбор признаков.

- Встроенные методы: Встроенные методы включают отбор признаков в сам алгоритм обучения. Эти методы определяют важность признаков в процессе обучения модели. В качестве примеров можно привести регрессию Лассо, регрессию Риджа и Random Forest.

- Гибридные методы: гибридные методы сочетают в себе сильные стороны методов фильтрации и обертывания. Они используют комбинацию статистических показателей и алгоритмов машинного обучения для выбора признаков.

`Корреляция Пирсона` - это мера линейной связи между двумя переменными. [-1,1]. Она предполагает, что обе переменные нормально распределены. Она измеряет только линейную корреляцию между переменными, игнорируя любые нелинейные связи.

`хи-квадрат` широко используется при отборе признаков для выявления наиболее информативных категориальных признаков. Он рассчитывает метрику хи-квадрат между целевой и категориальной переменной и выбирает нужное количество переменных с наилучшими значениями хи-квадрат.

`лассо` зануляет веса признаков которые линейно зависят с существующими

`Permutation importance`(определение важности признаков): Произвольным образом переставить или перетасовать один столбец в наборе датасета валидации, оставив все остальные столбцы нетронутыми. Признак считается «важным», если точность модели падает и его изменение вызывает увеличение ошибок. С другой стороны, признак считается «неважным», если перетасовка его значений не влияет на точность модели

`Библиотека SHAP` (SHapley Additive exPlanations) используется для объяснения работы моделей машинного обучения и интерпретации их предсказаний. Она предоставляет инструменты для вычисления значимости признаков в модели и позволяет понять, как каждый признак влияет на предсказания

# 2. Сделайте шаги из предыдущего упражнения по чтению данных из [kaggle](https://www.kaggle.com/competitions/two-sigma-connect-rental-listing-inquiries/data) и препроцессингу данных

Сначала настроим api Kaggle, что бы можно былдо читать напрямую из kaggle в Colab

In [151]:
import pandas as pd
import math
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
! pip install shap -q
import shap

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"olgabobkova","key":"e398d91d274c73eeb614e1a7bcd0723e"}'}

In [3]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle competitions download two-sigma-connect-rental-listing-inquiries

 88% 98.0M/112M [00:01<00:00, 51.6MB/s]
100% 112M/112M [00:01<00:00, 66.7MB/s] 


In [5]:
! unzip two-sigma-connect-rental-listing-inquiries.zip
! unzip train.json.zip
! unzip test.json.zip

Archive:  two-sigma-connect-rental-listing-inquiries.zip
  inflating: Kaggle-renthop.torrent  
  inflating: images_sample.zip       
  inflating: sample_submission.csv.zip  
  inflating: test.json.zip           
  inflating: train.json.zip          
Archive:  train.json.zip
  inflating: train.json              
Archive:  test.json.zip
  inflating: test.json               


## 2.1. Read all data.
Нам нужен только файл train.json

In [6]:
df = pd.read_json('train.json')
df.head(2)


,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low


Оставим только нужные столбцы

In [7]:
 # Оставляем пока только нужные столбцы
df = df[['bathrooms',
         'features',
         'bedrooms',
         'created',
         'price',
         'interest_level']].reset_index(drop=True)
df.head()

,bathrooms,features,bedrooms,created,price,interest_level
0,1.0,"[Dining Room, Pre-War, Laundry in Building, Di...",1,2016-06-16 05:55:27,2400,medium
1,1.0,"[Doorman, Elevator, Laundry in Building, Dishw...",2,2016-06-01 05:44:33,3800,low
2,1.0,"[Doorman, Elevator, Laundry in Building, Laund...",2,2016-06-14 15:19:59,3495,medium
3,1.5,[],3,2016-06-24 07:54:24,3000,medium
4,1.0,"[Doorman, Elevator, Fitness Center, Laundry in...",0,2016-06-28 03:50:23,2795,low


## 2.2. Preprocess “interest level” feature.

In [8]:
# Посмотрим униклаьные значения в столбце interest_level
df['interest_level'].value_counts()

,count
interest_level,
low,34284
medium,11229
high,3839


In [9]:
# interest _level переводим в категориальные данные
categorical_data = {'low': 0, 'medium':1, 'high':2}
df['interest_level'] = df['interest_level'].map(categorical_data)
df.head()

,bathrooms,features,bedrooms,created,price,interest_level
0,1.0,"[Dining Room, Pre-War, Laundry in Building, Di...",1,2016-06-16 05:55:27,2400,1
1,1.0,"[Doorman, Elevator, Laundry in Building, Dishw...",2,2016-06-01 05:44:33,3800,0
2,1.0,"[Doorman, Elevator, Laundry in Building, Laund...",2,2016-06-14 15:19:59,3495,1
3,1.5,[],3,2016-06-24 07:54:24,3000,1
4,1.0,"[Doorman, Elevator, Fitness Center, Laundry in...",0,2016-06-28 03:50:23,2795,0


## 2.3. Create features: 'Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed', 'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding', 'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck', 'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony', 'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace'

In [10]:
# в столбце features удалим все ненужные символы
def removing(string):
    temp = ','.join(string)
    temp = temp.replace(' ', '')
    return temp

df['features'] = df['features'].apply(lambda x: removing(x))
df.head()

,bathrooms,features,bedrooms,created,price,interest_level
0,1.0,"DiningRoom,Pre-War,LaundryinBuilding,Dishwashe...",1,2016-06-16 05:55:27,2400,1
1,1.0,"Doorman,Elevator,LaundryinBuilding,Dishwasher,...",2,2016-06-01 05:44:33,3800,0
2,1.0,"Doorman,Elevator,LaundryinBuilding,LaundryinUn...",2,2016-06-14 15:19:59,3495,1
3,1.5,,3,2016-06-24 07:54:24,3000,1
4,1.0,"Doorman,Elevator,FitnessCenter,LaundryinBuilding",0,2016-06-28 03:50:23,2795,0


In [11]:
features = ['Elevator',
                   'HardwoodFloors',
                   'CatsAllowed',
                   'DogsAllowed',
                   'Doorman',
                   'Dishwasher',
                   'NoFee',
                   'LaundryinBuilding',
                   'FitnessCenter',
                   'Pre-War',
                   'LaundryinUnit',
                   'RoofDeck',
                   'OutdoorSpace',
                   'DiningRoom',
                   'HighSpeedInternet',
                   'Balcony',
                   'SwimmingPool',
                   'LaundryInBuilding',
                   'NewConstruction',
                   'Terrace']

In [12]:
# Добавляем новые признаки и пока заполняем нулями
for feature in features:
    df[feature] = 0
df.head()

,bathrooms,features,bedrooms,created,price,interest_level,Elevator,HardwoodFloors,CatsAllowed,DogsAllowed,...,LaundryinUnit,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
0,1.0,"DiningRoom,Pre-War,LaundryinBuilding,Dishwashe...",1,2016-06-16 05:55:27,2400,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,"Doorman,Elevator,LaundryinBuilding,Dishwasher,...",2,2016-06-01 05:44:33,3800,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,"Doorman,Elevator,LaundryinBuilding,LaundryinUn...",2,2016-06-14 15:19:59,3495,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.5,,3,2016-06-24 07:54:24,3000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,"Doorman,Elevator,FitnessCenter,LaundryinBuilding",0,2016-06-28 03:50:23,2795,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
 # Выставялем значение 1 если данный признак есть в столбце features
for index, row in df.iterrows():
    for i in features:
        if i in row['features']:
            df.at[index, i] = 1

del df['features']
df.head()

,bathrooms,bedrooms,created,price,interest_level,Elevator,HardwoodFloors,CatsAllowed,DogsAllowed,Doorman,...,LaundryinUnit,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
0,1.0,1,2016-06-16 05:55:27,2400,1,0,1,1,1,0,...,0,0,0,1,0,0,0,0,0,0
1,1.0,2,2016-06-01 05:44:33,3800,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2,2016-06-14 15:19:59,3495,1,1,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,1.5,3,2016-06-24 07:54:24,3000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,2016-06-28 03:50:23,2795,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49352 entries, 0 to 49351
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bathrooms          49352 non-null  float64
 1   bedrooms           49352 non-null  int64  
 2   created            49352 non-null  object 
 3   price              49352 non-null  int64  
 4   interest_level     49352 non-null  int64  
 5   Elevator           49352 non-null  int64  
 6   HardwoodFloors     49352 non-null  int64  
 7   CatsAllowed        49352 non-null  int64  
 8   DogsAllowed        49352 non-null  int64  
 9   Doorman            49352 non-null  int64  
 10  Dishwasher         49352 non-null  int64  
 11  NoFee              49352 non-null  int64  
 12  LaundryinBuilding  49352 non-null  int64  
 13  FitnessCenter      49352 non-null  int64  
 14  Pre-War            49352 non-null  int64  
 15  LaundryinUnit      49352 non-null  int64  
 16  RoofDeck           493

# 3. Implement next methods

## 3.1. Split data into 2 parts randomly with parameter test_size (ratio from 0 to 1), return train and test samples


In [15]:
def my_train_test_split(df, test_size):
  assert test_size > 0 and test_size < 1, 'Значение test_size должно быть в диапазоне (0-1)'
  # перемешиваем строки pandas
  temp_shuffled = df.sample(frac=1)
  test_len = math.ceil(len(temp_shuffled) * test_size)
  train = temp_shuffled.iloc[test_len:,:]
  test = temp_shuffled.iloc[:test_len,:]
  return train, test


In [16]:
train, test = my_train_test_split(df, test_size=0.2)
print(f'Иссходный размер:{df.shape} \n train размер:{train.shape} \n  test размер:{test.shape}')

Иссходный размер:(49352, 25) 
 train размер:(39481, 25) 
  test размер:(9871, 25)


## 3.2. Split data into 3 parts randomly with parameters validation_size and test_size, return train, validation and test samples

In [17]:
def my_train_val_test_split(df, validation_size, test_size):
  assert test_size > 0 and test_size < 1, 'Значение test_size должно быть в диапазоне (0-1)'
  assert validation_size > 0 and validation_size < 1, 'Значение validation_size должно быть в диапазоне (0-1)'
  assert test_size + validation_size < 1, 'Сумма test_size и validation_size не должно превышать 1'
  # перемешиваем строки pandas
  temp_shuffled = df.sample(frac=1)
  test_len = math.ceil(len(temp_shuffled) * test_size)
  val_len = math.ceil(len(temp_shuffled) * validation_size)

  test = temp_shuffled.iloc[:test_len,:]
  val = temp_shuffled.iloc[test_len:val_len + test_len,:]
  train = temp_shuffled.iloc[val_len + test_len:,:]
  return train, val, test

In [18]:
train, val, test = my_train_val_test_split(df, test_size=0.2, validation_size=0.3)
print(f'Иссходный размер:{df.shape} \n train размер:{train.shape} \n val размер:{val.shape}\n  test размер:{test.shape}')

Иссходный размер:(49352, 25) 
 train размер:(24675, 25) 
 val размер:(14806, 25)
  test размер:(9871, 25)


## 3.3. Split data into 2 parts with parameter date_split, return train and test samples splitted by date_split param

In [19]:
def train_test_split_date(df, date_col, date_split):
    return df[df[date_col] < date_split], df[df[date_col] >= date_split]

In [20]:
train, test = train_test_split_date(df, 'created', '2016-05-23')

print(train.created.max())
print(test.created.min())

2016-05-22 21:03:18
2016-05-23 01:01:02


## 3.4. Split data into 3 parts with parameters validation_date and test_date, return train, validation and test samples splitted by input params

In [26]:
def train_validation_test_split_date(df, validation_date, test_date):
  train = df[df['created'] < validation_date]
  val = df[(df['created'] >= validation_date) & (df['created'] < test_date)]
  test = df[df['created'] >= test_date]
  return train, val, test

In [27]:
train, val, test = train_validation_test_split_date(df, '2016-05-23', '2016-06-29')

print('train min: ', train.created.min())
print('train max: ', train.created.max())
print('val min: ', val.created.min())
print('val max: ', val.created.max())
print('test min: ', test.created.min())
print('test max: ', test.created.max())

train min:  2016-04-01 22:12:41
train max:  2016-05-22 21:03:18
val min:  2016-05-23 01:01:02
val max:  2016-06-28 22:09:53
test min:  2016-06-29 01:10:29
test max:  2016-06-29 21:41:47


# 4. Implement next cross validation methods:

## 4.1. K-fold, where k is input parameter, return list of train and test indexes

In [35]:
def my_KFold(df, k):
  assert k > 1
  size = math.ceil(df.shape[0]/k) # размер фолда
  # res_list - хранит k+1 целых числа которые делят выборку на фолды
  res_list = [i for i in range(0, df.shape[0], size)]
  res_list.append(df.shape[0])
  result_arrays =[]
  for i in range(len(res_list)-1):
    # индексы тренировочной выборки
    result_arrays.append(np.array([i for i in range(0, res_list[i])]
                                  + [i for i in range(res_list[i+1], res_list[len(res_list)-1])]))
    # индексы тестовой выборки
    result_arrays.append(np.arange(res_list[i], res_list[i+1]))
  return result_arrays

## 4.2. Grouped K-fold, where k and group_field are input parameter, return list of train and test indexes

In [36]:
def my_GroupKFold(df, group_field, k=2):
    assert k > 1 and k <= df[group_field].nunique()
    df.sort_values(by=[group_field])
    group_list = []
    sum=0
    for i in df[group_field].value_counts():
        group_list.append(sum+i-1)
        sum += i
    res_list = [0]
    if k < df[group_field].nunique():
        size = math.ceil(df.shape[0]/k)
        size_list = [i for i in range(0, df.shape[0], size)]
        size_list.remove(0)
        for i in range(0, k-1):
            closest_value = min(group_list, key=lambda x: abs(x - size_list[i]))
            res_list.append(closest_value)
            group_list.remove(closest_value)
        res_list.append(df.shape[0])
    else:
        res_list += group_list

    result_arrays =[]
    for i in range(len(res_list)-1):
        result_arrays.append(np.array([i for i in range(0, res_list[i])]
                                      + [i for i in range(res_list[i+1], res_list[len(res_list)-1])])) # train
        result_arrays.append(np.arange(res_list[i], res_list[i+1]))  # test
    return result_arrays

## 4.3. Stratified K-fold, where k and stratify_field are input parameter, return list of train and test indexes

In [46]:
def my_StratifiedKFold(df, strata, k=2):
    assert k > 1
    dict_idxes = {}
    dict_sizes = {}
    for i in strata.unique():
        dict_idxes[i] = df.index[df['interest_level'] == i].tolist()
        dict_sizes[i] = round(df.index[df['interest_level'] == i].size/k)
    size = round(df.shape[0]/k)
    result_arrays =[]
    # print(res_list)
    for j in range(k):
        arr_train = (list([dict_idxes[i][:j*dict_sizes[i]] for i in dict_idxes.keys()]) +
                    list([dict_idxes[i][(j+1)*dict_sizes[i]:] for i in dict_idxes.keys()]))
        arr_train = list(itertools.chain(*arr_train))
        result_arrays.append(np.array(arr_train)) # train
        arr_test = list([dict_idxes[i][j*dict_sizes[i]:(j+1)*dict_sizes[i]] for i in dict_idxes.keys()])
        arr_test = list(itertools.chain(*arr_test))
        result_arrays.append(np.array(arr_test))  # test
    return result_arrays

## 4.4. Time series split, where k and date_field are input parameter, return list of train and test indexes

In [61]:
def my_TimeSeriesSplit(df, date_field=None, k=2):
    assert k > 1
    if date_field != None:
        df_sorted = df.sort_values(by=[date_field])
    else:
        df_sorted = df
    size = math.ceil(df.shape[0]/(k+1))
    result_arrays =[]
    sup = k
    for i in range(k):
        result_arrays.append(np.array(df_sorted.index[:(i+1)*size].tolist())) # train
        result_arrays.append(np.array(df_sorted.index[(i+1)*size:(i+2)*size].tolist()))  # test
    return result_arrays

# 5. Cross-validation comparison

## 5.1.
- Apply all implemented validation methods from above to our dataset.
- Apply the corresponding methods from sklearn.
- Compare the resulting distributions of features for the training part of the dataset between sklearn and your own implementation

### KFold

In [39]:
for i in my_KFold(df, 5):
    print(i, len(i))

[ 9871  9872  9873 ... 49349 49350 49351] 39481
[   0    1    2 ... 9868 9869 9870] 9871
[    0     1     2 ... 49349 49350 49351] 39481
[ 9871  9872  9873 ... 19739 19740 19741] 9871
[    0     1     2 ... 49349 49350 49351] 39481
[19742 19743 19744 ... 29610 29611 29612] 9871
[    0     1     2 ... 49349 49350 49351] 39481
[29613 29614 29615 ... 39481 39482 39483] 9871
[    0     1     2 ... 39481 39482 39483] 39484
[39484 39485 39486 ... 49349 49350 49351] 9868


In [41]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
for train, test in kf.split(df):
    print("%s %s\n%s %s" % (train, len(train), test, len(test)))

[ 9871  9872  9873 ... 49349 49350 49351] 39481
[   0    1    2 ... 9868 9869 9870] 9871
[    0     1     2 ... 49349 49350 49351] 39481
[ 9871  9872  9873 ... 19739 19740 19741] 9871
[    0     1     2 ... 49349 49350 49351] 39482
[19742 19743 19744 ... 29609 29610 29611] 9870
[    0     1     2 ... 49349 49350 49351] 39482
[29612 29613 29614 ... 39479 39480 39481] 9870
[    0     1     2 ... 39479 39480 39481] 39482
[39482 39483 39484 ... 49349 49350 49351] 9870


### Grouped KFold

In [43]:
for i in my_GroupKFold(df, 'interest_level', 3):
    print(i, len(i))

[34283 34284 34285 ... 49348 49349 49350] 15068
[    0     1     2 ... 34280 34281 34282] 34283
[    0     1     2 ... 49348 49349 49350] 38122
[34283 34284 34285 ... 45509 45510 45511] 11229
[    0     1     2 ... 45509 45510 45511] 45512
[45512 45513 45514 ... 49348 49349 49350] 3839


In [44]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=3)

for train, test in gkf.split(df, groups=df.interest_level):
    print("%s %s\n%s %s" % (train, len(train), test, len(test)))

[    0     2     3 ... 49349 49350 49351] 15068
[    1     4     5 ... 49345 49346 49347] 34284
[    1     4     5 ... 49346 49347 49351] 38123
[    0     2     3 ... 49348 49349 49350] 11229
[    0     1     2 ... 49348 49349 49350] 45513
[    7    17    31 ... 49312 49341 49351] 3839


### Stratified Kfold

In [57]:
for i in my_StratifiedKFold(df, df['interest_level'], 3):
    print(i, len(i))

[16804 16809 16814 ... 49312 49341 49351] 32901
[    0     2     3 ... 16332 16359 16377] 16451
[    0     2     3 ... 49312 49341 49351] 32901
[16804 16809 16814 ... 32871 32872 32887] 16451
[    0     2     3 ... 32871 32872 32887] 32902
[33087 33095 33097 ... 49312 49341 49351] 16450


In [58]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3) # 2 и более

for train_index, test_index in skf.split(df.drop(['interest_level'], axis=1), df['interest_level']):
    print("%s %s\n%s %s" % (train, len(train), test, len(test)))

[    0     1     2 ... 49348 49349 49350] 45513
[    7    17    31 ... 49312 49341 49351] 3839
[    0     1     2 ... 49348 49349 49350] 45513
[    7    17    31 ... 49312 49341 49351] 3839
[    0     1     2 ... 49348 49349 49350] 45513
[    7    17    31 ... 49312 49341 49351] 3839


### Time Series Split

In [63]:
for i in my_TimeSeriesSplit(df, 'created', 3):
    print(i, len(i))

[44454 46931 45569 ... 33984 42832 38871] 12338
[48900 46458 43079 ... 25422 20458 22549] 12338
[44454 46931 45569 ... 25422 20458 22549] 24676
[25023 21022 17813 ... 11600  2810 16820] 12338
[44454 46931 45569 ... 11600  2810 16820] 37014
[ 3072 12873  6226 ...   247 13840 11297] 12338


In [64]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)
for train, test in tscv.split(df):
    print("%s %s\n%s %s" % (train, len(train), test, len(test)))

[    0     1     2 ... 12335 12336 12337] 12338
[12338 12339 12340 ... 24673 24674 24675] 12338
[    0     1     2 ... 24673 24674 24675] 24676
[24676 24677 24678 ... 37011 37012 37013] 12338
[    0     1     2 ... 37011 37012 37013] 37014
[37014 37015 37016 ... 49349 49350 49351] 12338


## 5.4. Compare all validation schemes. Choose the best one. Explain your choice.

В данном случае нам нужно предсказать цену недвижимости.
Я бы не использовал `TimeSeriesSplit` поскольку это не временные ряды и диапазон дат не имеет большой разницы и не влияет на цену недвижимости.

Поскольку столбец `interest_level` имеет сущщественную значисость, я бы выбрал метод разделения `StratifiedSplit`

# 6. Feature selection

## 6.1. Fit lasso regression model with normalized features. Use your method for split samples into 3 parts by field created with ratio 60/20/20 – train/validation/test.

In [130]:
train, val, test = my_train_val_test_split(df, test_size=0.2, validation_size=0.2)

print(train.shape)
print(val.shape)
print(test.shape)

(29610, 25)
(9871, 25)
(9871, 25)


In [131]:
X_train = train.drop(["price", "created"], axis=1)
y_train = train['price']

X_val = val.drop(["price", "created"], axis=1)
y_val = val['price']

X_test = test.drop(["price", "created"], axis=1)
y_test = test['price']

print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

print(X_test.shape)
print(y_test.shape)

(29610, 23)
(29610,)
(9871, 23)
(9871,)
(9871, 23)
(9871,)


In [132]:
# Нормализуем данные только в двух столбцах
selected_columns = ['bathrooms', 'bedrooms']

scaler = StandardScaler().fit(X_train[selected_columns])

X_train_scl = X_train.copy()
X_train_scl[selected_columns] = scaler.transform(X_train[selected_columns])

X_val_scl = X_val.copy()
X_val_scl[selected_columns] = scaler.transform(X_val[selected_columns])

X_test_scl = X_test.copy()
X_test_scl[selected_columns] = scaler.transform(X_test[selected_columns])

X_test_scl.head(5)

,bathrooms,bedrooms,interest_level,Elevator,HardwoodFloors,CatsAllowed,DogsAllowed,Doorman,Dishwasher,NoFee,...,LaundryinUnit,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
7204,3.551768,2.206528,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,1
6027,-0.419978,-1.374518,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
22605,-0.419978,-1.374518,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9589,-0.419978,-0.479257,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19966,-0.419978,0.416005,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
# Обучим модель на Lasso

%%time
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
print(f"MAE:  {(mae):.2f}")
print(f"RMSE: {(rmse):.2f}")
print(f"R2:   {(r2):.5f}")

MAE:  1128.64
RMSE: 1965.98
R2:   0.38808
CPU times: user 109 ms, sys: 22.5 ms, total: 132 ms
Wall time: 82 ms


Занесем данные в таблицу

In [134]:
result_MAE = pd.DataFrame({'model': [], 'validation': [], 'test': []})
result_RMSE = pd.DataFrame({'model': [], 'validation': [], 'test': []})
result_R2 = pd.DataFrame({'model': [], 'validation': [], 'test': []})

In [135]:
result_MAE.loc[len(result_MAE)] = ['lasso_default', mae, mean_absolute_error(y_test, lasso.predict(X_test))]
result_RMSE.loc[len(result_RMSE)] = ['lasso_default', rmse, np.sqrt(mean_squared_error(y_test, lasso.predict(X_test)))]
result_R2.loc[len(result_R2)] = ['lasso_default', r2, r2_score(y_test, lasso.predict(X_test))]

## 6.2. Sort features by weight coefficients from model, refit model on top 10 features and compare quality.

Существует мнение что признаки имеющие наибольшие веса наиболее сильно вкладываются в предсказание. И действительно, если только взять 10 из 23 признаков у которых наибольшие веса слегка улучшают показатели качества модели

In [136]:
%%time
sorted_indices = np.argsort(lasso.coef_)
top_10_indices = sorted_indices[-10:]

X_train_top_10 = X_train.iloc[:, top_10_indices]
X_val_top_10 = X_val.iloc[:, top_10_indices]
X_test_top_10 = X_test.iloc[:, top_10_indices]

lasso_top_10 = Lasso()
lasso_top_10.fit(X_train_top_10, y_train)

val_r2_top10 = r2_score(y_val, lasso_top_10.predict(X_val_top_10))
val_rmse_top10 = np.sqrt(mean_squared_error(y_val, lasso_top_10.predict(X_val_top_10)))
val_mae_top10 = mean_absolute_error(y_val, lasso_top_10.predict(X_val_top_10))

print(f"MAE:  {(val_mae_top10):.2f}")
print(f"RMSE: {(val_rmse_top10):.2f}")
print(f"R2:   {(val_r2_top10):.5f}")

MAE:  1044.74
RMSE: 1928.67
R2:   0.41109
CPU times: user 56.2 ms, sys: 5.98 ms, total: 62.2 ms
Wall time: 68.6 ms


Занесем данные в таблицу

In [137]:
result_MAE.loc[len(result_MAE)] = ['lasso_weight_top10',
                                   val_mae_top10,
                                 mean_absolute_error(y_test, lasso_top_10.predict(X_test_top_10))]
result_RMSE.loc[len(result_RMSE)] = ['lasso_weight_top10',
                                     val_rmse_top10,
                                   np.sqrt(mean_squared_error(y_test, lasso_top_10.predict(X_test_top_10)))]
result_R2.loc[len(result_R2)] = ['lasso_weight_top10',
                                 val_r2_top10,
                                     r2_score(y_test, lasso_top_10.predict(X_test_top_10))]

## 6.3. Implement method for simple feature selection by nan ration in feature and correlation. Apply this method for the feature set and take top 10 features, refit model and measure quality.

In [138]:
# Посмотрим насколько коррелирует price с признаками
cor_matrix = df.corr(numeric_only=True)
cor_matrix = cor_matrix['price']
cor_matrix = cor_matrix.drop(['price'], axis=0)
cor_matrix

,price
bathrooms,0.069661
bedrooms,0.051788
interest_level,-0.023783
Elevator,0.023479
HardwoodFloors,0.001520
CatsAllowed,0.011639
DogsAllowed,0.013062
Doorman,0.033875
Dishwasher,0.013154
NoFee,0.005229


In [139]:
# Отсортируем строки и избавимся от знака минус.
sorted_correlation = cor_matrix.abs().sort_values(ascending=False)
sorted_correlation

,price
bathrooms,0.069661
bedrooms,0.051788
Doorman,0.033875
interest_level,0.023783
Elevator,0.023479
LaundryinUnit,0.022481
DiningRoom,0.021228
FitnessCenter,0.015368
Terrace,0.014562
Dishwasher,0.013154


In [140]:
selected_features = sorted_correlation[:10].index
selected_features

Index(['bathrooms', 'bedrooms', 'Doorman', 'interest_level', 'Elevator',
       'LaundryinUnit', 'DiningRoom', 'FitnessCenter', 'Terrace',
       'Dishwasher'],
      dtype='object')

In [141]:
%%time

X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]
X_test_selected = X_test[selected_features]

lasso_corr = Lasso()
lasso_corr.fit(X_train_selected, y_train)

val_r2_top10 = r2_score(y_val, lasso_corr.predict(X_val_selected))
val_rmse_top10 = np.sqrt(mean_squared_error(y_val, lasso_corr.predict(X_val_selected)))
val_mae_top10 = mean_absolute_error(y_val, lasso_corr.predict(X_val_selected))

print(f"MAE:  {(val_mae_top10):.2f}")
print(f"RMSE: {(val_rmse_top10):.2f}")
print(f"R2:   {(val_r2_top10):.5f}")

MAE:  1078.36
RMSE: 1935.48
R2:   0.40693
CPU times: user 1.06 s, sys: 7.8 ms, total: 1.06 s
Wall time: 1.11 s


Занесем данные в таблицу

In [142]:
result_MAE.loc[len(result_MAE)] = ['lasso_corr_top10', val_mae_top10,
                                 mean_absolute_error(y_test, lasso_corr.predict(X_test_selected))]
result_RMSE.loc[len(result_RMSE)] = ['lasso_corr_top10', val_rmse_top10,
                                   np.sqrt(mean_squared_error(y_test, lasso_corr.predict(X_test_selected)))]
result_R2.loc[len(result_R2)] = ['lasso_corr_top10', val_r2_top10,
                                     r2_score(y_test, lasso_corr.predict(X_test_selected))]

# 6.4. Implement permutation importance method and take top 10 features, refit model and measure quality.

In [143]:
def my_permutation_importance(model, X_test, y_test):
    n_features = X_test.shape[1]
    importances = np.zeros(n_features)

    for i in range(n_features):
        X_test_permuted = X_test.copy()
        X_test_permuted.iloc[:, i] = np.random.permutation(X_test_permuted.iloc[:, i])

        score_orig = model.score(X_test, y_test)
        score_permuted = model.score(X_test_permuted, y_test)

        importances[i] = score_orig - score_permuted
    return importances

In [144]:
%%time
importances = my_permutation_importance(Lasso().fit(X_train, y_train), X_val, y_val)
top_10_indices = np.argsort(importances)[-10:]
X_train_top_10 = X_train.iloc[:, top_10_indices]
X_val_top_10 = X_val.iloc[:, top_10_indices]
X_test_top_10 = X_test.iloc[:, top_10_indices]

model_top_10 = Lasso().fit(X_train_top_10, y_train)

val_r2_top10 = r2_score(y_val, model_top_10.predict(X_val_top_10))
val_rmse_top10 = np.sqrt(mean_squared_error(y_val, model_top_10.predict(X_val_top_10)))
val_mae_top10 = mean_absolute_error(y_val, model_top_10.predict(X_val_top_10))

print(f"MAE:  {(val_mae_top10):.2f}")
print(f"RMSE: {(val_rmse_top10):.2f}")
print(f"R2:   {(val_r2_top10):.5f}")

MAE:  1074.31
RMSE: 1937.88
R2:   0.40545
CPU times: user 2.18 s, sys: 340 ms, total: 2.52 s
Wall time: 2.52 s


Занесем данные в таблицу

In [145]:
result_MAE.loc[len(result_MAE)] = ['lasso_permutation_top10', val_mae_top10,
                                 mean_absolute_error(y_test, model_top_10.predict(X_test_top_10))]
result_RMSE.loc[len(result_RMSE)] = ['lasso_permutation_top10', val_rmse_top10,
                                   np.sqrt(mean_squared_error(y_test, model_top_10.predict(X_test_top_10)))]
result_R2.loc[len(result_R2)] = ['lasso_permutation_top10', val_r2_top10,
                                     r2_score(y_test, model_top_10.predict(X_test_top_10))]

## 6.5. Import shap and also refit model on top 10 features

In [146]:
%%time

background_data = X_train.iloc[0, :].values.reshape(1, -1)
explainer = shap.KernelExplainer(model=lasso.predict, data=background_data, link="identity")

instance = X_train.iloc[0, :].values.reshape(1, -1)
shap_values = explainer.shap_values(instance)

feature_importances = np.mean(np.abs(shap_values), axis=0)

top_10_indices = np.argsort(feature_importances)[-10:]
X_top_10 = X_train.iloc[:, top_10_indices]
X_top_10_val = X_val.iloc[:, top_10_indices]
X_top_10_test = X_test.iloc[:, top_10_indices]

lasso_top_10 = Lasso()
lasso_top_10.fit(X_top_10, y_train)
val_r2_top10 = r2_score(y_val, lasso_top_10.predict(X_top_10_val))
val_rmse_top10 = np.sqrt(mean_squared_error(y_val, lasso_top_10.predict(X_top_10_val)))
val_mae_top10 = mean_absolute_error(y_val, lasso_top_10.predict(X_top_10_val))

print(f"MAE:  {(val_mae_top10):.2f}")
print(f"RMSE: {(val_rmse_top10):.2f}")
print(f"R2:   {(val_r2_top10):.5f}")

  0%|          | 0/1 [00:00<?, ?it/s]

MAE:  1235.51
RMSE: 2185.23
R2:   0.24399
CPU times: user 148 ms, sys: 16.7 ms, total: 164 ms
Wall time: 168 ms


Занесепм данные в таблицу

In [147]:
result_MAE.loc[len(result_MAE)] = ['lasso_shap_top10', val_mae_top10,
                                 mean_absolute_error(y_test, lasso_top_10.predict(X_top_10_test))]
result_RMSE.loc[len(result_RMSE)] = ['lasso_shap_top10', val_rmse_top10,
                                   np.sqrt(mean_squared_error(y_test, lasso_top_10.predict(X_top_10_test)))]
result_R2.loc[len(result_R2)] = ['lasso_shap_top10', val_r2_top10,
                                     r2_score(y_test, lasso_top_10.predict(X_top_10_test))]

## 6.6. Compare the quality of these methods for different sides – speed, metrics and stability

In [148]:
result_MAE

,model,validation,test
0,lasso_default,1128.636988,1157.553615
1,lasso_weight_top10,1044.744001,1069.268864
2,lasso_corr_top10,1078.355723,1111.851560
3,lasso_permutation_top10,1074.309329,1106.705063
4,lasso_shap_top10,1235.505792,1268.888741


In [149]:
result_RMSE

,model,validation,test
0,lasso_default,1965.982917,2327.424894
1,lasso_weight_top10,1928.669991,2285.085373
2,lasso_corr_top10,1935.475000,2298.256533
3,lasso_permutation_top10,1937.884828,2301.841598
4,lasso_shap_top10,2185.225766,2519.118816


In [150]:
result_R2

,model,validation,test
0,lasso_default,0.388082,0.292413
1,lasso_weight_top10,0.411089,0.317923
2,lasso_corr_top10,0.406926,0.310037
3,lasso_permutation_top10,0.405448,0.307883
4,lasso_shap_top10,0.243991,0.171055


- Касательно стабильности по метрике R2 между тестовой и валидационной оказалась схема с SHAP, но в то же время это самая плохая метрика
- Лучашая метрика оказалась у схемы где мы отбираем признаки по наибольшему весу.
- Так же схема по отбору признаков по значению вессового коэффициента оказалась самой шустрой

# 7. Hyperparameters optimization

## 7.1. Implement grid search and random search methods for alpha and l1_ratio for ElasticNet model from sklearn.
## 7.2. Find the best combination for the model hyperparameters.

In [187]:
%%time
elastic_net = ElasticNet()

param_grid = {
    'alpha': [0.1, 0.5, 1.0],
    'l1_ratio': [0.25, 0.5, 0.75]
}

grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

CPU times: user 11.3 s, sys: 1.61 s, total: 12.9 s
Wall time: 10.4 s


GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 0.5, 1.0],
                         'l1_ratio': [0.25, 0.5, 0.75]})

In [188]:
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'alpha': 0.5, 'l1_ratio': 0.75}


In [189]:
%%time

param_dist = {
    'alpha': [0.1, 0.5, 1.0],
    'l1_ratio': [0.25, 0.5, 0.75]
}

random_search = RandomizedSearchCV(estimator=elastic_net, param_distributions=param_dist, n_iter=5, cv=5)
random_search.fit(X_train, y_train)

CPU times: user 2.76 s, sys: 978 ms, total: 3.74 s
Wall time: 1.98 s


RandomizedSearchCV(cv=5, estimator=ElasticNet(), n_iter=5,
                   param_distributions={'alpha': [0.1, 0.5, 1.0],
                                        'l1_ratio': [0.25, 0.5, 0.75]})

In [190]:
# Вывод наилучших гиперпараметров и оценки
print("Best Hyperparameters:", random_search.best_params_)

Best Hyperparameters: {'l1_ratio': 0.75, 'alpha': 0.5}


## 7.3. Fit resulted model.

In [191]:
elastic_net_final = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net_final.fit(X_train, y_train)

ElasticNet(alpha=0.1)

Оценим метрики


In [192]:
y_pred_val = elastic_net_final.predict(X_val)
y_pred_test = elastic_net_final.predict(X_test)

r2_elastic_test = r2_score(y_test, y_pred_test)
rmse_elastic_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_elastic_test = mean_absolute_error(y_test, y_pred_test)

r2_elastic_val = r2_score(y_val, y_pred_val)
rmse_elastic_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
mae_elastic_val = mean_absolute_error(y_val, y_pred_val)

In [193]:
hyper_MAE = pd.DataFrame({'model': [], 'validation': [], 'test': []})
hyper_RMSE = pd.DataFrame({'model': [], 'validation': [], 'test': []})
hyper_R2 = pd.DataFrame({'model': [], 'validation': [], 'test': []})

hyper_MAE.loc[len(hyper_MAE)] = ['elastic', mae_elastic_val, mae_elastic_test]
hyper_RMSE.loc[len(hyper_RMSE)] = ['elastic', rmse_elastic_val, rmse_elastic_test]
hyper_R2.loc[len(hyper_R2)] = ['elastic', r2_elastic_val, r2_elastic_test]

## 7.4. Import optuna and configure the same experiment with ElasticNet.

In [194]:
! pip install optuna -q
import optuna

In [195]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0.001, 1.0, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)

    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    return mse

In [196]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-08-19 16:36:29,077] A new study created in memory with name: no-name-122a160e-6364-496c-8329-2d071b3f9cad
[I 2024-08-19 16:36:29,133] Trial 0 finished with value: 3800279.034174906 and parameters: {'alpha': 0.1336796471186513, 'l1_ratio': 0.9259175617543389}. Best is trial 0 with value: 3800279.034174906.
[I 2024-08-19 16:36:29,266] Trial 1 finished with value: 3783166.5297542755 and parameters: {'alpha': 0.020385593200271727, 'l1_ratio': 0.3521078516542253}. Best is trial 1 with value: 3783166.5297542755.
[I 2024-08-19 16:36:29,435] Trial 2 finished with value: 3857769.857921813 and parameters: {'alpha': 0.0040099463176754505, 'l1_ratio': 0.5208569338884939}. Best is trial 1 with value: 3783166.5297542755.
[I 2024-08-19 16:36:29,661] Trial 3 finished with value: 3870843.7737541553 and parameters: {'alpha': 0.002050061228250392, 'l1_ratio': 0.7782409105989571}. Best is trial 1 with value: 3783166.5297542755.
[I 2024-08-19 16:36:29,765] Trial 4 finished with value: 3785690.40248

In [197]:
best_params = study.best_params

print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'alpha': 0.11598197805095745, 'l1_ratio': 0.3541811391167403}


In [198]:
elastic_optuna_final = ElasticNet(alpha=0.11598, l1_ratio=0.35418)
elastic_optuna_final.fit(X_train, y_train)

ElasticNet(alpha=0.11598, l1_ratio=0.35418)

In [199]:
y_pred_val = elastic_optuna_final.predict(X_val)
y_pred_test = elastic_optuna_final.predict(X_test)

r2_elastic_test = r2_score(y_test, y_pred_test)
rmse_elastic_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_elastic_test = mean_absolute_error(y_test, y_pred_test)

r2_elastic_val = r2_score(y_val, y_pred_val)
rmse_elastic_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
mae_elastic_val = mean_absolute_error(y_val, y_pred_val)

In [200]:
hyper_MAE.loc[len(hyper_MAE)] = ['elastic_optuna', mae_elastic_val, mae_elastic_test]
hyper_RMSE.loc[len(hyper_RMSE)] = ['elastic_optuna', rmse_elastic_val, rmse_elastic_test]
hyper_R2.loc[len(hyper_R2)] = ['elastic_optuna', r2_elastic_val, r2_elastic_test]

## 7.5. Run optuna on one of the cross validation schemes

In [201]:
elastic_optuna_final.coef_

array([1411.22341726,  717.79868266, -594.35902983,  578.23102389,
       -265.41298688,  185.69089141,  339.96693312, 1285.34309693,
       -135.09234057, -293.57487817, -431.59591346, -356.57918447,
        -54.98556195,  365.74353926, -120.79592626, -117.54610827,
        177.90194649, -159.78581778,  -26.44718835,   16.65316009,
       -315.78661704,  -31.36335898,  176.3810009 ])

In [202]:
%%time
sorted_indices = np.argsort(elastic_optuna_final.coef_)
top_10_indices = sorted_indices[-10:]

X_train_top_10 = X_train.iloc[:, top_10_indices]
X_val_top_10 = X_val.iloc[:, top_10_indices]
X_test_top_10 = X_test.iloc[:, top_10_indices]

optuna_top_10 = ElasticNet(alpha=0.11598, l1_ratio=0.35418)
optuna_top_10.fit(X_train_top_10, y_train)

y_pred_val = optuna_top_10.predict(X_val_top_10)
y_pred_test = optuna_top_10.predict(X_test_top_10)

r2_elastic_test = r2_score(y_test, y_pred_test)
rmse_elastic_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_elastic_test = mean_absolute_error(y_test, y_pred_test)

r2_elastic_val = r2_score(y_val, y_pred_val)
rmse_elastic_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
mae_elastic_val = mean_absolute_error(y_val, y_pred_val)

CPU times: user 303 ms, sys: 2.91 ms, total: 306 ms
Wall time: 314 ms


In [203]:
hyper_MAE.loc[len(hyper_MAE)] = ['elastic_optuna_top10', mae_elastic_val, mae_elastic_test]
hyper_RMSE.loc[len(hyper_RMSE)] = ['elastic_optuna_top10', rmse_elastic_val, rmse_elastic_test]
hyper_R2.loc[len(hyper_R2)] = ['elastic_optuna_top10', r2_elastic_val, r2_elastic_test]

## 7.6. Estimate metrics and compare these approaches.

In [204]:
hyper_MAE

,model,validation,test
0,elastic,1036.070223,1060.042340
1,elastic_optuna,1013.453751,1036.527477
2,elastic_optuna_top10,997.693818,1020.962009


In [205]:
hyper_RMSE

,model,validation,test
0,elastic,1920.715449,2281.037371
1,elastic_optuna,1917.920491,2277.004795
2,elastic_optuna_top10,1934.930074,2289.164916


In [206]:
hyper_R2

,model,validation,test
0,elastic,0.415936,0.320337
1,elastic_optuna,0.417635,0.322738
2,elastic_optuna_top10,0.407259,0.315485
